In [ ]:
# Install Required Libraries
!pip install --quiet diffusers transformers accelerate safetensors gradio einops

In [ ]:
# Import Libraries and Check GPU
import torch
from diffusers import StableDiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Load the Uncensored Stable Diffusion Pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype,
    safety_checker=None
).to(device)

In [ ]:
# Generate Image from Prompt
prompt = "sports trophy"
image = pipe(prompt).images[0]
image.save("output.png")
image


In [ ]:
# Load Zeroscope and Generate Video
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "cerspense/zeroscope_v2_576w",
    torch_dtype=torch.float32
).to("cpu")

prompt = "flying "
video_frames = pipe(prompt, num_inference_steps=25).frames


In [ ]:
import numpy as np
import imageio
from IPython.display import HTML
from base64 import b64encode

# Extract actual frames from the batch
video_frames_np = video_frames[0]  # shape: (16, 256, 256, 3)
video_frames_np = (video_frames_np * 255).astype(np.uint8)  # convert float32 -> uint8

# Save video
video_path = "/content/generated_video.mp4"
imageio.mimsave(video_path, video_frames_np, fps=8)

# Display the video in Colab
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=400 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")
